<a href="https://colab.research.google.com/github/tomonari-masada/course2022-nlp/blob/main/07_PyTorch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch入門 (3)
* IMDbデータセットの感情分析をPyTorchで実装する。
 * 前にscikit-learnを使って同じ作業をおこなった。

## (A) fasttextの単語埋め込みを使ったモデル

* データとしては、以前作ったIMDbの文書埋め込みを使う。
 * この文書埋め込みは、fasttextの単語埋め込みをもとに作られていた。

### 準備

* あらかじめランタイムのタイプをGPUに設定しておこう。

In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

np.random.seed(123)
torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

### 事前にfasttextでベクトル化されたIMDbデータを読み込む
 * 下記のリンク先にある`.npy`ファイルを、あらかじめ自分のGoogle Driveの適当な場所に置いておく。
  * https://drive.google.com/drive/folders/1wSoIzSbZ2UqGQowiVDBI20h_A3hQNbtV?usp=sharing

In [ ]:
PATH = '/content/drive/MyDrive/2022Courses/nlp/imdb/'

texts = {}
labels = {}
for tag in ['train', 'test']:
  with open(f'{PATH}{tag}.npy', 'rb') as f:
    texts[tag] = np.load(f)
  with open(f'{PATH}{tag}_labels.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [ ]:
for tag in ['train', 'test']:
  print(tag, texts[tag].shape)

* PyTorchのテンソルに変換しておく。

In [ ]:
for tag in ['train', 'test']:
  texts[tag] = torch.tensor(texts[tag])
  labels[tag] = torch.tensor(labels[tag])

### Dataset

In [ ]:
class MyDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]

In [ ]:
train_dataset = MyDataset(texts['train'], labels['train'])
test_dataset = MyDataset(texts['test'], labels['test'])

valid_size = len(train_dataset) // 5
train_size = len(train_dataset) - valid_size
split_train_, split_valid_ = random_split(train_dataset, [train_size, valid_size])

### DataLoader

In [ ]:
BATCH_SIZE = 64

train_loader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(split_valid_, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

### モデルの定義

In [ ]:
class TextSentiment(nn.Module):
  def __init__(self, emsize, num_class):
    super(TextSentiment, self).__init__()
    self.fc1 = nn.Linear(emsize, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
EMSIZE = texts['train'].size(1) # 埋め込みベクトルの次元
NUM_CLASS = len(np.unique(labels['train'])) # クラスの個数

model = TextSentiment(EMSIZE, NUM_CLASS).to(device)

In [ ]:
print(f"embedding dim: {EMSIZE}, number of classes: {NUM_CLASS}")

### 損失関数と最適化アルゴリズム

* 損失関数を除いて、以下の設定はいい加減なので、自分で調整してみよう。
* schedulerの使い方は、調べてみよう。

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20,50], gamma=0.1)

### 訓練を行なう関数

In [ ]:
def train(dataloader):
  model.train()
  total_acc = 0.0
  total_count = 0
  for input, target in dataloader:
    input, target = input.to(device), target.to(device)
    output = model(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_acc += (output.argmax(1) == target).sum().item()
    total_count += len(target) # 表示用の集計
  return total_acc / total_count

### 評価を行なう関数
* 正解率で評価する関数を定義しておく。

In [ ]:
def evaluate(dataloader):
  model.eval()
  total_acc = 0.0
  total_count = 0
  for input, target in dataloader:
    with torch.no_grad():
      input, target = input.to(device), target.to(device)
      output = model(input)
      total_acc += (output.argmax(1) == target).sum().item()
      total_count += len(target)
  return total_acc / total_count

### 訓練と評価の実施

In [ ]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
  epoch_start_time = time.time()
  train_acc = train(train_loader)
  valid_acc = evaluate(valid_loader)
  print(f'epoch {epoch:3d} | '
        f'time: {time.time() - epoch_start_time:5.2f}s | '
        f'train accuracy {train_acc:8.3f} | '
        f'valid accuracy {valid_acc:8.3f}')

* training lossとvalidation lossの差が大きいと、generalizeしない。
* 以下、各自試行錯誤してください。

* ハイパーパラメータのチューニングが済んだら、テストセットで評価する。

In [ ]:
test_acc = evaluate(test_loader)
print(f'test accuracy {test_acc:8.3f}')

## (B) 単語埋め込みもパラメータになっているモデル
* fasttextの単語埋め込みを使うのをやめる。
* 単語埋め込みも、モデルのパラメータと同時に学習することにする。

* 下記のPyTorch公式のチュートリアルも参照。
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* ただし、`torchtext`の仕組みを利用すると、語彙集合の作成がとても遅い。
 * `torchtext`のトークナイザがとても遅い。
* そこで、語彙集合の作成は、自分で行う。

### IMDbデータセットをテキストデータとして読み直す

In [ ]:
!pip install ml_datasets

In [ ]:
from ml_datasets import imdb

train_data, test_data = imdb()
train_texts, train_labels = zip(*train_data)
test_texts, test_labels = zip(*test_data)

In [ ]:
train_texts[0]

In [ ]:
train_labels[0]

* ラベルを0/1の整数に変換しておく。

In [ ]:
label_id = { "pos":1, "neg":0 }

train_labels = [label_id[label] for label in train_labels]
test_labels = [label_id[label] for label in test_labels]

In [ ]:
print(train_labels[:10])

### sklearnのCountVectorizerを使ってトークン化

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=10, max_df=0.2)
vectorizer.fit(train_texts)

In [ ]:
vocabulary = vectorizer.get_feature_names_out()
print(vocabulary[:10])

In [ ]:
len(vocabulary)

### `OrderedDict`から`torchtext`の語彙集合を作る
* 作り方は下記のリンク先を参照。
 * https://pytorch.org/text/stable/vocab.html#id1

In [ ]:
from torchtext.vocab import vocab
from collections import OrderedDict

# OrderedDictを作成
# keyは単語、valueは何でもいい（ここでは1にした）
vocab_ordered_dict = OrderedDict(zip(vocabulary, np.ones(len(vocabulary))))

# 未知語は全て"<unk>"という特殊なトークンへ置き換えることにする
unknown_token = "<unk>"

# OrderedDictをもとにtorchtextでの語彙集合を作成
imdb_vocab = vocab(vocab_ordered_dict, specials=[unknown_token])

# 語彙にない単語のインデックスは全て"<unk>"と同じインデックスになるよう、設定する
imdb_vocab.set_default_index(imdb_vocab[unknown_token])

In [ ]:
# 語彙数は"<unk>"の分だけ元より多くなる
len(imdb_vocab)

* ある単語が語彙集合に入っているかどうかは、下のようにしてチェックできる。
 * インデックス0が返ってきたら、未知語として扱われているということ。

In [ ]:
imdb_vocab(["is", "efjwsdnd"])

In [ ]:
imdb_vocab(["apple", "machine"])

### テキストをインデックスの列へ変換する関数を定義

In [ ]:
# fit済みのCountVectorizerから、前処理とトークナイザを持ってくる
preprocessor = vectorizer.build_preprocessor()
tokenizer = vectorizer.build_tokenizer()

# 前処理、トークナイザ、インデックス列への変換を、一つの処理としてまとめる
text_pipeline = lambda x: imdb_vocab(tokenizer(preprocessor(x)))

In [ ]:
text = "This is a pen."
print(preprocessor(text))
print(tokenizer(preprocessor(text)))
print(text_pipeline(text))

### Dataset

In [ ]:
class MyTextDataset(Dataset):
  def __init__(self, labels, texts):
    self.labels = labels
    self.texts = texts

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, index):
    return self.labels[index], self.texts[index]

train_dataset = MyTextDataset(train_labels, train_texts)
test_dataset = MyTextDataset(test_labels, test_texts)

In [ ]:
valid_size = len(train_dataset) // 5
train_size = len(train_dataset) - valid_size
test_size = len(test_dataset)

split_train_, split_valid_ = random_split(train_dataset, [train_size, valid_size])

In [ ]:
train_dataset[0]

### DataLoader

* ミニバッチがオフセットの情報を含むようにする。
 * オフセットは、ミニバッチに含まれるシーケンスの切れ目を表す。

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
  label_list, text_list, offsets = [], [], [0]
  for (_label, _text) in batch:
    label_list.append(_label)
    processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
    text_list.append(processed_text)
    offsets.append(processed_text.size(0))
  label_list = torch.tensor(label_list, dtype=torch.int64)
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  text_list = torch.cat(text_list)
  return label_list.to(device), text_list.to(device), offsets.to(device)

In [ ]:
BATCH_SIZE = 64
train_loader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_loader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [ ]:
# オフセットを表すテンソルに注目。
next(iter(train_loader))

### モデルの定義
* 下記ページのまま。
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* `EmbeddingBag`レイヤは、単語をまずembedし、シーケンス内で平均を計算してくれる。
 * https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html

In [ ]:
class TextClassificationModel(nn.Module):
  
  def __init__(self, vocab_size, embed_dim, num_class):
    super(TextClassificationModel, self).__init__()
    self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
    self.fc = nn.Linear(embed_dim, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.embedding.weight.data.uniform_(-initrange, initrange)
    self.fc.weight.data.uniform_(-initrange, initrange)
    self.fc.bias.data.zero_()

  def forward(self, text, offsets):
    embedded = self.embedding(text, offsets)
    return self.fc(embedded)

In [ ]:
NUM_CLASS = len(label_id)
VOCAB_SIZE = len(imdb_vocab)
EMSIZE = 64
model = TextClassificationModel(VOCAB_SIZE, EMSIZE, NUM_CLASS).to(device)

### 損失関数と最適化アルゴリズム

In [ ]:
EPOCHS = 10 # epoch
LR = 0.001  # learning rate

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

### 訓練を行なう関数

In [ ]:
def train(dataloader):
  model.train()
  total_acc, total_count = 0, 0
  for idx, (label, text, offsets) in enumerate(dataloader):
    optimizer.zero_grad()
    predicted_label = model(text, offsets)
    loss = criterion(predicted_label, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
    optimizer.step()
    total_acc += (predicted_label.argmax(1) == label).sum().item()
    total_count += label.size(0)
  return total_acc / total_count

### 評価を行なう関数

In [ ]:
def evaluate(dataloader):
  model.eval()
  total_acc, total_count = 0, 0
  with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(dataloader):
      predicted_label = model(text, offsets)
      loss = criterion(predicted_label, label)
      total_acc += (predicted_label.argmax(1) == label).sum().item()
      total_count += label.size(0)
  return total_acc / total_count

### 訓練と評価の実施

In [ ]:
for epoch in range(1, EPOCHS + 1):
  epoch_start_time = time.time()
  train_acc = train(train_loader)
  valid_acc = evaluate(valid_loader)
  print(f'epoch {epoch:3d} | '
        f'time: {time.time() - epoch_start_time:5.2f}s | '
        f'train accuracy {train_acc:8.3f} | '
        f'valid accuracy {valid_acc:8.3f}')

* 検証セット上での評価値でチューニングしてから、テストセットで最終評価。

In [ ]:
print(f"test accuracy {evaluate(test_loader):8.3f}")

# 課題
* モデルやoptimizerやschedulerを変更して、validation setを使ってチューニングしよう。
* 最後に、自分で選択した設定を使って、test set上で評価しよう。